In [1]:
# 장르 속성을 이용한 영화 콘텐츠 기반 필터링
# 콘텐츠 기반 필터링은 사용자가 특정 영화를 좋아했다면,
# 해당 영화와 유사한 특성(장르, 구성 요소 등)을 가진 다른 영화를 추천하는 방식입니다.

import pandas as pd  # 데이터 조작과 분석을 위한 라이브러리
import numpy as np   # 수치 계산 및 배열 연산을 위한 라이브러리
import warnings      # 경고 메시지를 제어하기 위한 라이브러리

warnings.filterwarnings('ignore')  # 경고 메시지 출력을 무시하도록 설정, 시각적 노이즈를 줄임

# CSV 파일('tmdb_5000_movies.csv')을 읽어 DataFrame('movies')으로 로드
# 경로('./tmdb_5000_movies.csv')는 현재 작업 디렉토리에 해당 파일이 있다고 가정
movies = pd.read_csv('./tmdb_5000_movies.csv')

print(movies.shape)  # DataFrame의 크기(행 개수, 열 개수)를 출력하여 데이터 규모를 확인
movies.head(2)       # 상위 2개의 행을 출력하여 DataFrame의 기본 구조를 미리 확인

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [2]:
# 검색, 추천 등에 자주 활용되는 컬럼만 선택하여 새 DataFrame 생성
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]
# 'id': 영화 식별자
# 'title': 영화 제목
# 'genres': 영화 장르 정보
# 'vote_average': 영화에 대한 평균 평점
# 'vote_count': 평점을 남긴 투표자 수
# 'popularity': 영화의 인기 지수
# 'keywords': 영화와 관련된 주요 키워드
# 'overview': 영화의 간략한 줄거리/개요

In [3]:
# pandas의 컬럼 표시 설정(max_colwidth)을 100으로 지정
# 데이터 시각 확인 시, genres나 keywords와 같이 문자열 정보가 긴 컬럼을 보다 쉽게 확인 가능
pd.set_option('max_colwidth', 100)

# 'movies_df'의 'genres'와 'keywords' 열만 확인 (상위 1행)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [4]:
from ast import literal_eval

# 'genres'와 'keywords' 열의 값들은 문자열 형태의 리스트/딕셔너리 표현으로 되어 있습니다.
# literal_eval 함수를 사용하여 문자열을 실제 파이썬 객체(list, dict 등)로 변환합니다.
# 예: "[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}]" → [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}]
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [5]:
# 각 영화의 'genres'와 'keywords' 열에 대해, 내부 리스트의 요소 중 'name' 값만 추출하여 리스트로 재구성
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])

# 'genres', 'keywords' 컬럼의 상위 1행을 확인해, 변환 결과를 확인
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해, 리스트 형태의 장르 정보를
# 공백으로 구분된 단어 형태의 문자열로 변환하여 'genres_literal' 열에 저장
# 예: ['Action', 'Adventure'] → "Action Adventure"
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))

# CountVectorizer 객체 생성
# - min_df=0: 용어가 등장하지 않는 문서가 있더라도 해당 용어를 모두 고려
# - ngram_range=(1, 2): 단어 1개(uni-gram)와 2개(bi-gram)로 나누어 빈도수를 계산
count_vect = CountVectorizer(min_df=0.0, ngram_range=(1, 2))

# 'genres_literal' 열에 대해 단어 빈도 벡터화(행렬 변환) 수행
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])

# 벡터화된 장르 행렬의 크기를 출력, (문서 수, 속성(단어) 수)
print(genre_mat.shape)

(4803, 276)


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# 'genre_mat' 행렬에 대해 코사인 유사도(cosine similarity)를 계산하여,
# 각 영화 간의 유사도 정보를 담은 행렬(genre_sim)을 생성
genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)  # (영화 개수, 영화 개수) 형태로, 각 영화 쌍 간의 장르 유사도 확인 가능
print(genre_sim[:1])    # 첫 번째 영화와 모든 영화 간의 유사도 값을 보여줌

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [8]:
# 코사인 유사도 행렬 'genre_sim'에서 각 영화가 유사도 기준 내림차순 정렬된 인덱스를 구함
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

print(genre_sim_sorted_ind[:1])  # 첫 번째 영화와의 유사도 순 정렬 결과 확인

[[   0 3494  813 ... 3038 3037 2401]]


In [9]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    # 'df' 중에서 'title' 열이 'title_name'과 동일한 행을 찾음
    title_movie = df[df['title'] == title_name]

    # 찾은 행의 index 번호를 획득하고, 'sorted_ind'에서 이 index에 해당하는
    # 상위 'top_n'개 유사 영화의 index 목록 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    # 2차원 배열 형태로 되어 있는 'similar_indexes'를 1차원 배열로 변경
    print(similar_indexes)  # 선택된 영화들의 index 확인용
    similar_indexes = similar_indexes.reshape(-1)

    # 추출된 index에 해당하는 영화를 DataFrame 형태로 반환
    return df.iloc[similar_indexes]

In [10]:
# 장르 속성을 이용한 영화 콘텐츠 기반 필터링

# 'The Godfather'와 유사한 영화를 찾아 추천하는 함수 호출
# - df: 영화 데이터가 담긴 DataFrame
# - sorted_ind: 각 영화의 유사도 기준으로 정렬된 인덱스 배열
# - 'The Godfather': 유사 영화를 찾을 기준이 되는 영화 제목
# - top_n=10: 추천할 유사 영화의 개수
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)

# 추천된 영화들의 'title'과 'vote_average' 컬럼만 선택하여 출력
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [11]:
# movies_df에서 'title', 'vote_average', 'vote_count' 컬럼만 선택하고,
# vote_average(평점) 기준으로 내림차순 정렬한 후 상위 10개 영화 추출
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [19]:
# 전체 영화의 평균 평점을 구함
C = movies_df['vote_average'].mean()

# 상위 40%에 해당하는 투표 수 임계값을 구함 (하위 60% 지점의 투표 수)
m = movies_df['vote_count'].quantile(0.6)

# 계산된 C와 m 값을 소수점 3자리까지 반올림하여 출력
print('C:', round(C, 3), 'm:', round(m, 3))

C: 6.092 m: 370.2


In [20]:
# 가중 평균 계산을 위한 기준값 설정
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)  # 최소 투표 수 기준값 (60분위수)
C = movies_df['vote_average'].mean()  # 전체 영화의 평균 평점

def weighted_vote_average(record):
    v = record['vote_count']     # 개별 영화의 투표 수
    R = record['vote_average']   # 개별 영화의 평균 평점
    
    # IMDB 가중 평점 계산 공식 적용
    return (((v/(v+m)) * R) + ((m/(m+v)) * C))

# 각 영화에 대해 가중 평점을 계산하여 'weighted_vote' 컬럼에 추가
movies['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

In [22]:
# 현재 사용 중인 DataFrame 이름 확인
print("DataFrame 이름:", [var for var in globals() if isinstance(globals()[var], pd.DataFrame)])

# 데이터 확인
print("\n기존 컬럼:", movies_df.columns.tolist())

# 가중 평점 계산 및 추가
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    return (((v/(v+m)) * R) + ((m/(m+v)) * C))

# movies_df에 weighted_vote 컬럼 추가 확인
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
print("\n추가된 컬럼 확인:", movies_df.columns.tolist())

# 결과 확인
result = movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]
print("\n상위 10개 영화:", result)

DataFrame 이름: ['_', '__', '___', 'movies', '_1', 'movies_df', '_3', '_5', 'similar_movies', '_10', '_11']

기존 컬럼: ['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview', 'genres_literal']

추가된 컬럼 확인: ['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview', 'genres_literal', 'weighted_vote']

상위 10개 영화:                          title  vote_average  weighted_vote  vote_count
1881  The Shawshank Redemption           8.5       8.396052        8205
3337             The Godfather           8.4       8.263591        5893
662                 Fight Club           8.3       8.216455        9413
3232              Pulp Fiction           8.3       8.207102        8428
65             The Dark Knight           8.2       8.136930       12002
1818          Schindler's List           8.3       8.126069        4329
3865                  Whiplash           8.3       8.123248        4254
809               Forrest Gump           8.2 

In [23]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    similar_indexes = sorted_ind[title_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending = False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]
   

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
